In [4]:
# import packages
import pandas as pd
import pickle as pkl
import os

In [5]:
train_file_dir =  '/Users/girishhanumantha/Documents/GitHub/adds-rr-demo-modeling/data/processed/rr_demo_features_train_h1.pkl'

In [6]:
# read training data
df_train = pd.read_pickle(train_file_dir)

In [7]:
df_grp = df_train.groupby(['demo_category'])

In [8]:
for grp in df_grp.groups:
    print(grp)
    print(len(df_grp.get_group(grp).dropna()))
    print(len(df_grp.get_group(grp)))
    print(len(df_grp.get_group(grp).dropna())/len(df_grp.get_group(grp)))

Age
100
90508
0.0011048747072082026
Core-Cume
100
90310
0.0011072970878086591
Gender
30
81762
0.00036691861745064946
Race
21
79074
0.000265574019273086
Total
50
45254
0.0011048747072082026


In [9]:
cols_pop = pd.DataFrame(columns=['col_name', 'perc_avl'])
for col in df_train.columns:
    cols_temp = pd.DataFrame([(col, len(df_train[~pd.isna(df_train[col])])/(1.0*(len(df_train))))], columns=['col_name', 'perc_avl'])
    cols_pop = pd.concat([cols_pop, cols_temp])

In [10]:
cols_pop.sort_values(by=['perc_avl'])

,col_name,perc_avl
0,pop_omt,0.064716
0,med_pop_prior,0.857853
0,mean_pop_artist_prior,0.857853
0,mr_pop_prior,0.857853
0,max_pop_prior,0.857853
...,...,...
0,diff_song_univ_spins_prior,1.000000
0,mr_market_spins_spins,1.000000
0,feat_artist_song,1.000000
0,diff_artist_univ_spins_prior,1.000000


In [15]:
df_train[(df_train['demo_category'] == 'Gender')][['segment_Female_(18-24)', 'segment_Female_Other']].sum()

segment_Female_(18-24)    40881
segment_Female_Other      40881
dtype: int64

In [16]:
score_file_dir =  '/Users/girishhanumantha/Documents/GitHub/adds-rr-demo-modeling/data/processed/rr_demo_features_score_h1.pkl'

In [27]:
# read scoring data
df_score = pd.read_pickle(score_file_dir)

In [28]:
df_score.describe()

,mediabase_id,station_id,breakout_id,pop_co,pop_omt,spins_am_pm_dr,format_spins,song_univ_spins,min_pop_prior,diff_spins_song_station_prior,...,segment_Core,segment_Female_(18-24),segment_Female_Other,segment_Hispanic,segment_Non-Core,segment_Old,segment_Total,segment_White,segment_Young,omt_co_flag_CO_only
count,5.487000e+04,5.487000e+04,54870.000000,4379.000000,0.0,54870.000000,54870.000000,54870.000000,54303.000000,54870.000000,...,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000,54870.000000
mean,2.240036e+06,3.349295e+06,272167.166466,77.962092,NaN,4.227957,173.260379,429.407035,73.719518,-0.202096,...,0.109550,0.087644,0.087644,0.057500,0.109459,0.109550,0.109550,0.100875,0.109550,0.079807
std,5.032721e+05,1.934679e+05,183108.087922,12.880727,NaN,11.068909,415.030587,932.831265,15.391633,3.705250,...,0.312331,0.282778,0.282778,0.232797,0.312217,0.312331,0.312331,0.301166,0.312331,0.270996
min,1.085550e+06,3.321797e+06,-2.000000,12.000000,NaN,0.000000,0.000000,0.000000,1.000000,-59.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.828077e+06,3.322025e+06,1.000000,70.000000,NaN,0.000000,9.000000,25.000000,63.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.394316e+06,3.322828e+06,400803.000000,79.000000,NaN,0.000000,33.000000,82.000000,73.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.691407e+06,3.323415e+06,401515.000000,87.000000,NaN,3.000000,86.000000,283.000000,85.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.848773e+06,4.762077e+06,428785.000000,142.000000,NaN,75.000000,2716.000000,5310.000000,135.000000,44.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
df_train['week_dt'].drop_duplicates().sort_values(ascending=False)

157      2022-11-27
313      2022-11-20
793      2022-11-13
312      2022-11-06
9        2022-10-30
            ...    
34230    2021-01-10
34813    2021-01-03
4522     2020-12-27
1844     2020-12-06
1821     2020-11-29
Name: week_dt, Length: 97, dtype: object

In [31]:
set(df_train.columns)-(set(df_score.columns))

{'omt_co_flag_OMT_CO', 'omt_co_flag_OMT_only'}

In [33]:
list(df_train.columns)

['mediabase_id',
 'station_id',
 'week_dt',
 'breakout_id',
 'demo_category',
 'pop_co',
 'pop_omt',
 'gcr',
 'gcr_adj',
 'artist_count',
 'count_pop_artist_prior',
 'mr_spins_song_station_prior',
 'song_station_weeks_since_last_spins',
 'max_pop_artist_prior',
 'spins_am_pm_dr',
 'min_pop_artist_prior_unv',
 'diff_spins_artist_station_prior',
 'market_spins',
 'avg_spins_song_station_prior',
 'mr_spins_song_market_prior',
 'avg_market_artist_spins_prior',
 'market_artist_spins',
 'format_spins',
 'song_station_weeks_since_first_spins',
 'song_last_test_co_weeks',
 'total_market_spins_prior',
 'per_diff_market_spins_spins_prior',
 'diff_spins_song_station_prior',
 'mr_pop_artist_prior_unv',
 'spins_total',
 'mean_pop_prior',
 'mr_artist_univ_spins',
 'total_spins_non_on_song_station_prior',
 'avg_song_univ_spins_prior',
 'mean_pop_artist_prior_unv',
 'per_diff_song_univ_spins_prior',
 'total_station_artist_spins_prior',
 'song_last_test_any_weeks',
 'total_artist_univ_spins_prior',
 'a